In [ ]:
import pandas as pd
import geopandas as gpd
import requests as r
import json

from IPython.display import SVG, display

In [ ]:
ontology = pd.read_csv("app/data/ontology.csv", index_col=0)
graduates = pd.read_csv("app/data/graduates.csv", index_col=0)
vacancy = pd.read_parquet("app/data/vacancy.gzip")
responses = pd.read_parquet("app/data/responses.gzip")
cv = pd.read_parquet("app/data/cv.gzip")
cities = gpd.read_file("app/data/cities.geojson")

In [ ]:
test_request = {
  "workforce_type": "all",
  "specialities": {'0': 0.9, '1': 1, '4': 0.1, '8': 0.5, '10': 0.9, '11': 0.2, '14': 1, '16': 1, '17': 0.6, '18': 0.5, '22': 1},
  "edu_groups": {
    "3": 0.1,
    "5": 0.5,
    "12": 0.7,
    "20": 0.5,
    "21": 0.6
    
  }
}

In [ ]:
# responses = r.post('http://10.32.1.62:5500/api/calculation/estimates', json=test_request)
# responses = r.post('http://172.17.0.2:5000/api/calculation/estimates', json=test_request)
responses = r.post('http://0.0.0.0:5000/api/calculation/estimates', json=test_request)

In [ ]:
gpd.GeoDataFrame.from_features(responses.json()['estimates']).columns

In [ ]:
estimates = gpd.GeoDataFrame.from_features(responses.json()['estimates'])
links = gpd.GeoDataFrame.from_features(responses.json()['links'])

In [ ]:
estimates[['region_city', "estimate", "num_in_migration"]]

In [ ]:
test_request = {
  "city": "Пермский край, Пермь"
}

# responses = r.post('http://10.32.1.62:5500/api/calculation/connection', json=test_request)
# responses = r.post('http://172.17.0.2:5000/api/calculation/connection', json=test_request)
responses = r.post('http://0.0.0.0:5000/api/calculation/connection', json=test_request)

In [ ]:
migration_link = gpd.GeoDataFrame.from_features(responses.json()['migration_link'])

In [ ]:
agglomeration_links = gpd.GeoDataFrame.from_features(responses.json()['agglomeration_links'])
agglomeration_nodes = gpd.GeoDataFrame.from_features(responses.json()['agglomeration_nodes'])

In [ ]:
agglomeration_nodes

In [ ]:
test_request = {
  "city_name": "Пермский край, Пермь",
  "plot": True,
  'estimates_table': json.loads(estimates.to_json())
}

# responses = r.post('http://10.32.1.62:5500/api/calculation/prediction', json=test_request)
# responses = r.post('http://172.17.0.2:5000/api/calculation/prediction', json=test_request)
responses = r.post('http://0.0.0.0:5000/api/calculation/prediction', json=test_request)

In [ ]:
city_features = responses.json()['city_features']
gpd.GeoDataFrame.from_features(city_features)[["estimate_after", "estimate_before"]]

In [ ]:
new_links = gpd.GeoDataFrame.from_features(responses.json()['new_links'])
new_links.set_index("cluster_center_cv").equals(migration_link.set_index("cluster_center_cv").loc[new_links["cluster_center_cv"]])

In [ ]:
slice = estimates["region_city"] == "Пермский край, Пермь"

In [ ]:
from IPython.display import SVG, display
display(SVG(responses.json()['plot']))

In [ ]:
estimates.columns

In [ ]:
estimates.loc[slice, 'vacancy_count'] -= 100

In [ ]:
estimates.loc[slice, 'median_salary'] += 10000
estimates.loc[slice, 'ueqi_social_and_leisure_infrastructure'] += 10

In [ ]:
test_request = {
  "city_name": "Пермский край, Пермь",
  "plot": True,
  'estimates_table': json.loads(estimates.to_json())
}

# responses = r.post('http://10.32.1.62:5500/api/calculation/prediction', json=test_request)
# responses = r.post('http://172.17.0.2:5000/api/calculation/prediction', json=test_request)
responses = r.post('http://0.0.0.0:5000/api/calculation/prediction', json=test_request)

In [ ]:
city_features = responses.json()['city_features']
gpd.GeoDataFrame.from_features(city_features)[["estimate_after", "estimate_before", "num_in_migration_before", "num_in_migration_after"]]

In [ ]:
from IPython.display import SVG, display
from matplotlib import pyplot as plt
display(SVG(responses.json()['plot']))

In [ ]:
response = r.get('http://0.0.0.0:5000/api/calculation/plots')

In [ ]:
response.json().keys()

In [ ]:
from IPython.display import SVG, display
display(SVG(response.json()['small']))

In [ ]:
from IPython.display import SVG, display
display(SVG(response.json()['average']))

In [ ]:
from IPython.display import SVG, display
display(SVG(response.json()['big']))

In [ ]:
from IPython.display import SVG, display
display(SVG(response.json()['large']))

In [ ]:
from IPython.display import SVG, display
display(SVG(response.json()['largest']))